### Problem 3

In [1]:
import numpy as np
from scipy.stats import norm

In [2]:
class GBM:

    def __init__(self,S0,r,sigma=None):
        self.S0=S0
        self.r=r
        self.sigma=sigma


In [3]:
class Call:

    def __init__(self,K,T):
        self.K=K
        self.T=T

    #This solution did not end up using this class.
    #But alternative solutions could reuse Black-Scholes code from the HW, which may make use of a Call class

In [4]:
class CallOnAverage:

    def __init__(self,K,T0,T1):
        self.K = K
        self.T0 = T0
        self.T1 = T1

In [5]:
class MCengine:

    def __init__(self,M,seed,useControl):
        self.M = M  #number of paths
        self.rng = np.random.default_rng(seed=seed) # Seeding the random number generator with a specified number helps make the calculations reproducible
        self.useControl = useControl

    def randomLogreturn(self,dynamics,deltat):
        return (dynamics.r-dynamics.sigma**2/2)*deltat + dynamics.sigma*np.sqrt(deltat)*self.rng.normal(size=self.M)

    def price_CallOnAverage_GBM(self,contract,dynamics):

        S05 = dynamics.S0 * np.exp(self.randomLogreturn(dynamics,contract.T0))
        S10 = S05 * np.exp(self.randomLogreturn(dynamics,contract.T1-contract.T0))
        zeropayoff = np.zeros(self.M)
        payoffdiscounted = np.exp(-dynamics.r*contract.T1) * np.maximum((S05+S10)/2 - contract.K,zeropayoff)

        if not self.useControl:
            return(np.mean(payoffdiscounted), np.std(payoffdiscounted)/np.sqrt(self.M))

        controlpayoff = np.exp(-dynamics.r*contract.T1) * np.maximum(S10 - contract.K,zeropayoff)
        F = dynamics.S0*np.exp(dynamics.r*contract.T1)
        std = dynamics.sigma*np.sqrt(contract.T1)
        d1 = np.log(F/contract.K)/std + std/2
        d2 = d1 - std
        controlexpectation = np.exp(-dynamics.r*contract.T1) * (F*norm.cdf(d1) - contract.K*norm.cdf(d2))
        covmatrix = np.cov(payoffdiscounted,controlpayoff)
        beta = covmatrix[0,1]/covmatrix[1,1]
        Ycv = payoffdiscounted + beta *(controlexpectation - controlpayoff)
        print('correlation:',np.corrcoef(payoffdiscounted, controlpayoff)[0,1])
        return(np.mean(Ycv), np.std(Ycv)/np.sqrt(self.M))


In [6]:
p3dynamics = GBM(sigma=0.70,S0=10,r=0.02)

In [7]:
p3contract = CallOnAverage(K=12,T0=0.5,T1=1.0)

In [8]:
p3aMC = MCengine(M=100000,seed=0,useControl=False)
(price3a, standard_error3a) = p3aMC.price_CallOnAverage_GBM(p3contract,p3dynamics)
print('price:',price3a, 'stderr:',standard_error3a)

price: 1.5735749676914146 stderr: 0.013133445215084201


In [9]:
p3bMC = MCengine(M=100000,seed=0,useControl=True)
(price3b, standard_error3b) = p3bMC.price_CallOnAverage_GBM(p3contract,p3dynamics)
print('price:',price3b, 'stderr:',standard_error3b)

correlation: 0.9395339840925245
price: 1.5661493307125633 stderr: 0.0044976229113177614
